# 4MULTIFACTORIAL: OPTIMIZACIÓN MULTIFACTORIAL AVANZADA

Este notebook demuestra el funcionamiento del módulo 4multifactorial, que implementa optimización Top-Down para construir carteras con exposiciones objetivo a factores específicos.

In [1]:
import sys
import importlib
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importar módulos (los nombres que empiezan con números requieren importlib)
datos = importlib.import_module('1datos')
markowitz = importlib.import_module('2markowitz')
factores = importlib.import_module('3factores')
multifactorial = importlib.import_module('4multifactorial')

print("Librerías importadas correctamente")

Librerías importadas correctamente


In [2]:
# Cargar y preparar datos
ruta_csv = '../data/prod_long_sharpe_u50_20260116_v5_train_dataset.csv'
retornos = datos.cargar_retornos(ruta_csv)
preparador = datos.PreparadorDatos(retornos, rf_anual=0.02)
preparador.calcular_estadisticas()
mu, Sigma, rf = preparador.obtener_estadisticas()

# Construir señales y matriz X
stats = datos.calcular_estadisticas_basicas(retornos)
momentum = factores.calcular_momentum_12_2(retornos)
mom_actual = momentum.iloc[-1].dropna()
volatilidades = factores.calcular_volatilidad_rolling(retornos)
vol_actual_63d = volatilidades['vol_63d'].iloc[-1].dropna()
betas = factores.calcular_betas(retornos)

senales = {
    'momentum': mom_actual,
    'vol_63d': -vol_actual_63d,
    'beta': -betas,
    'sharpe_hist': stats['sharpe_historico']
}
senales_norm = factores.normalizar_senales(senales)
X, nombres_factores = factores.construir_matriz_caracteristicas(senales_norm)

print(f"Datos preparados: μ shape {mu.shape}, X shape {X.shape}")

Datos cargados: 1760 días, 50 activos


ValueError: Data must be 1-dimensional, got ndarray of shape (1760, 50) instead

## Paso 1: Definir Exposiciones Objetivo y Pesos

In [3]:
# Exposiciones objetivo base
exposiciones_objetivo = {
    'momentum': 0.5,
    'vol_63d': -0.3,
    'beta': 0.0,
    'sharpe_hist': 0.4
}

b_star = np.array([exposiciones_objetivo[f] for f in nombres_factores])
print("Exposiciones objetivo b*:")
for i, nombre in enumerate(nombres_factores):
    print(f"  {nombre}: {b_star[i]:.2f}")

# Calcular pesos de exposición
W_k = multifactorial.calcular_pesos_exposicion(X, metodo='inverso_varianza')
print(f"\nPesos de exposición W_k:")
for i, nombre in enumerate(nombres_factores):
    print(f"  {nombre}: {W_k[i]:.4f}")

NameError: name 'nombres_factores' is not defined

In [ ]:
cartera_topdown = multifactorial.optimizar_topdown(
    mu, Sigma, rf, X, b_star, W_k,
    lambda_riesgo=2.0, tau_rotacion=0.5
)

if cartera_topdown:
    print("Cartera Top-Down Multifactorial:")
    print(f"  Sharpe: {cartera_topdown['sharpe']:.4f}")
    print(f"  Rentabilidad: {cartera_topdown['rentabilidad']*100:.2f}%")
    print(f"  Volatilidad: {cartera_topdown['volatilidad']*100:.2f}%")
    print(f"  Tracking Error: {cartera_topdown['tracking_error']:.4f}")
    print(f"\n  Exposiciones:")
    for i, nombre in enumerate(nombres_factores):
        print(f"    {nombre:15s}: objetivo={b_star[i]:6.2f}, real={cartera_topdown['exposiciones_reales'][i]:6.2f}")

## Paso 3: Estrategias Alternativas

In [ ]:
# Estrategia 1: High Momentum + Low Vol
exp_mom_lowvol = multifactorial.crear_estrategia_momentum_lowvol(nombres_factores)
cart_mom_lowvol = multifactorial.optimizar_topdown(mu, Sigma, rf, X, exp_mom_lowvol, W_k)

# Estrategia 2: Quality
exp_quality = multifactorial.crear_estrategia_quality(nombres_factores)
cart_quality = multifactorial.optimizar_topdown(mu, Sigma, rf, X, exp_quality, W_k)

# Estrategia 3: Minimum Variance
exp_minvar = multifactorial.crear_estrategia_min_variance(nombres_factores)
cart_minvar = multifactorial.optimizar_topdown(mu, Sigma, rf, X, exp_minvar, W_k,
                                 lambda_riesgo=10.0, tau_rotacion=0.1)

# Benchmark: Max Sharpe Markowitz
cartera_max_sharpe = markowitz.optimizar_sharpe_maximo(mu, Sigma, rf)

print("Comparación de Estrategias:")
estrategias = {
    'Max Sharpe (Markowitz)': cartera_max_sharpe,
    'Top-Down Multifactorial': cartera_topdown,
    'High Mom + Low Vol': cart_mom_lowvol,
    'Quality': cart_quality,
    'Min Variance': cart_minvar
}

for nombre, cart in estrategias.items():
    if cart:
        print(f"\n{nombre}:")
        print(f"  Sharpe: {cart['sharpe']:.4f}")
        print(f"  Ret: {cart['rentabilidad']*100:.2f}%")
        print(f"  Vol: {cart['volatilidad']*100:.2f}%")

## Paso 4: Visualizar Comparación

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sharpe por estrategia
nombres = [n for n, c in estrategias.items() if c]
sharpes = [c['sharpe'] for n, c in estrategias.items() if c]

axes[0].barh(nombres, sharpes)
axes[0].set_xlabel('Sharpe Ratio')
axes[0].set_title('Sharpe Ratio por Estrategia')
axes[0].grid(True, alpha=0.3)

# Riesgo vs Rentabilidad
rentabilidades = [c['rentabilidad']*100 for n, c in estrategias.items() if c]
volatilidades = [c['volatilidad']*100 for n, c in estrategias.items() if c]

axes[1].scatter(volatilidades, rentabilidades, s=200, alpha=0.7)
for i, nombre in enumerate(nombres):
    axes[1].annotate(nombre, (volatilidades[i], rentabilidades[i]), 
                     fontsize=8, ha='center')
axes[1].set_xlabel('Volatilidad (%)')
axes[1].set_ylabel('Rentabilidad (%)')
axes[1].set_title('Riesgo vs Rentabilidad')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/modulo4_comparacion.png', dpi=300, bbox_inches='tight')
plt.show()

## Resumen del módulo 4multifactorial

El módulo 4multifactorial ha completado exitosamente:

1. Definición de exposiciones objetivo a factores
2. Cálculo de pesos de exposición (inverso de varianza)
3. Optimización Top-Down con tracking de exposiciones
4. Comparación de múltiples estrategias (Momentum+LowVol, Quality, MinVar)
5. Comparación con benchmark de máximo Sharpe

Las carteras optimizadas están listas para validación en el módulo 5validacion.